# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1105],
        [0.1640]], grad_fn=<AddmmBackward0>)

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3057,  0.0117, -0.0598, -0.1237, -0.1858, -0.3463, -0.1517, -0.2138]])), ('bias', tensor([0.2875]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0887], requires_grad=True)
tensor([0.0887])


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [4]:
net[2].weight.grad == None

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示。


In [6]:
net.state_dict()['2.bias'].data

tensor([0.0887])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2596],
        [0.2596]], grad_fn=<AddmmBackward0>)

[**设计了网络后，我们看看它是如何工作的。**]


In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.1999, -0.4073, -0.1200, -0.2033, -0.1573,  0.3546, -0.2141, -0.2483])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0214, -0.0015, -0.0100, -0.0058]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5236,  0.0516, -0.3236,  0.3794])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


同样，我们实现了一个`my_init`函数来应用到`net`。


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[5.4079, 9.3334, 5.0616, 8.3095],
        [0.0000, 7.2788, -0.0000, -0.0000]], grad_fn=<SliceBackward0>)

注意，我们始终可以直接设置参数。


In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.3334,  6.0616,  9.3095])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
这里有一个问题：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层
（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/1829)


In [4]:
from torch import nn


class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


model = NestMLP()

print(model.state_dict())

# 访问net层的参数
print("访问net层的参数")
for name, param in model.net.named_parameters():
    print(f"Parameter name: {name}, shape:{param.shape}, Parameter: {param}")

# 访问linear层的参数
print("访问linear层的参数")
for name, param in model.linear.named_parameters():
    print(f"Parameter name: {name}, shape:{param.shape}, Parameter: {param}")

OrderedDict([('net.0.weight', tensor([[-0.0871,  0.1496,  0.1944,  ...,  0.0842,  0.0624,  0.2090],
        [ 0.1811,  0.1548,  0.2151,  ...,  0.0051, -0.2210,  0.0961],
        [-0.1303,  0.1426,  0.1708,  ..., -0.2018, -0.1239,  0.2141],
        ...,
        [ 0.2082, -0.0442, -0.2019,  ...,  0.0275,  0.1364,  0.0231],
        [-0.1905, -0.1747,  0.1127,  ...,  0.2128, -0.1570, -0.0088],
        [-0.1377,  0.1600, -0.0388,  ...,  0.0991,  0.1483,  0.0908]])), ('net.0.bias', tensor([-0.0200,  0.0640,  0.1445, -0.1835,  0.1116,  0.0058,  0.1419, -0.1064,
        -0.2217, -0.0994,  0.1923,  0.0461, -0.2180,  0.2036,  0.0394,  0.1842,
         0.2057, -0.0666, -0.2150,  0.0215,  0.1013,  0.1038, -0.0241,  0.0633,
        -0.1958,  0.0556,  0.0523,  0.1173,  0.1361,  0.1965, -0.1361,  0.0741,
        -0.1856, -0.2152,  0.2102,  0.1648, -0.1108, -0.0681,  0.1988, -0.0069,
         0.1253, -0.1915, -0.0716,  0.1921, -0.0287,  0.1678,  0.2139, -0.0002,
         0.0559, -0.0074, -0.1180,  0.1

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 模型参数
input_size = 4
hidden_size = 8
output_size = 4
lr = 0.01
epochs = 2

# 构建带有共享参数层的多层感知机
shared_fc = nn.Linear(hidden_size, hidden_size)
MLP = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    shared_fc,
    nn.ReLU(),
    shared_fc,
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),
)

# 训练数据
X = torch.randn(1, input_size)
Y = torch.randn(1, output_size)
# 优化器
optimizer = optim.SGD(MLP.parameters(), lr=lr)
# 训练模型
for epoch in range(epochs):
    # 前向传播和计算损失
    Y_pred = MLP(X)
    loss = nn.functional.mse_loss(Y_pred, Y)
    # 反向传播和更新梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 打印每层的参数和梯度
    for name, param in MLP.named_parameters():
        print(name, param.data, param.grad)
    print("Epoch: {}, Loss: {}".format(epoch, loss.item()))

0.weight tensor([[ 0.1002,  0.2945,  0.2631,  0.4947],
        [ 0.1769, -0.4392, -0.4046, -0.0505],
        [-0.3955,  0.1895, -0.1194,  0.1817],
        [ 0.3997, -0.4676, -0.4291,  0.2868],
        [ 0.1132, -0.0339,  0.1908, -0.1005],
        [-0.3307,  0.1292, -0.3551, -0.1775],
        [-0.0328, -0.3282, -0.3184, -0.0806],
        [-0.0080, -0.2942,  0.0720, -0.0845]]) tensor([[ 0.0047,  0.0084,  0.0285, -0.0429],
        [ 0.0010,  0.0018,  0.0063, -0.0094],
        [-0.0026, -0.0047, -0.0160,  0.0240],
        [ 0.0063,  0.0113,  0.0384, -0.0577],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0103,  0.0185,  0.0627, -0.0943],
        [-0.0000, -0.0000, -0.0000,  0.0000]])
0.bias tensor([ 0.0188,  0.0897, -0.0593, -0.1048,  0.1556, -0.1515, -0.2471, -0.3818]) tensor([-0.0244, -0.0054,  0.0137, -0.0328,  0.0000,  0.0000, -0.0537,  0.0000])
2.weight tensor([[ 0.3422, -0.2100,  0.2800,  0.1959,  0.0147,  0.3409,  0.1944,  0.

为什么共享参数是个好主意？

解答：

  1. 节约内存：共享参数可以减少模型中需要存储的参数数量，从而减少内存占用。

  2. 加速收敛：共享参数可以让模型更加稳定，加速收敛。

  3. 提高泛化能力：共享参数可以帮助模型更好地捕捉数据中的共性，提高模型的泛化能力。

  4. 加强模型的可解释性：共享参数可以让模型更加简洁明了，加强模型的可解释性。

